In [1]:
import numpy as np
import random
import pandas as pd

In [2]:
NUM_PLAYERS = 3
random.seed(7)

In [3]:
# helper functions
def randSelectPlayer(candidates, exclude=None):
    choices = list(candidates)
    if exclude:
        choices.remove(exclude)
    
    return random.choice(choices)

def give(giver, receiver, playerWealth):
    v = playerWealth[giver]
    w = playerWealth[receiver]
    amt = min(v, w)
    playerWealth[receiver] += amt
    playerWealth[giver] -= amt
    return playerWealth, amt
    
def getPlayerByWealth(playerWealth, amt):
    for p, w in playerWealth.items():
        if w == amt:
            return p
        
    return None
    

def checkGameStatus(playerWealth, winAmt, loseAmt=0):
    """
    Checks if there are any winners and losers.
    """
    winner = getPlayerByWealth(playerWealth, winAmt)
    loser = getPlayerByWealth(playerWealth, loseAmt)        
    
    return winner, loser

def reportGameResult(winner, losers, numRounds):
    # prints losers in order
    for i in range(len(losers)):
        print(f"Loser {i+1}: {losers[i]}")
    print(f"Winner: {winner}")
    print(f"Number of rounds played: {numRounds}")

def reportGameProgress(numRounds, giver, receiver, transferAmt, playerWealth):
    print(f"Round {numRounds}:")
    print(f"Player {giver} gave Player {receiver} ${transferAmt}")
    print(f"Remaining players' wealth: {playerWealth}\n")

In [4]:
def playGame(x, y, z, printProgress=False, printResult=False):
    # initialise
    playerWealth = {1: x, 2: y, 3: z}
    numPlayers = NUM_PLAYERS
    total = x + y + z
    winner = None
    losers = []  # losers in order - might be interested in, e.g. P(Player 2 loses before 1)
    numRounds = 0
    
    while numPlayers > 1:
        giver = randSelectPlayer(playerWealth)
        receiver = randSelectPlayer(playerWealth, exclude=giver)
        playerWealth, transferAmt = give(giver, receiver, playerWealth)
        
        winner, loser = checkGameStatus(playerWealth, total)
        if loser:
            losers.append(loser)
            numPlayers -= 1
            playerWealth.pop(loser)
        
        numRounds += 1
        
        if printProgress:
            reportGameProgress(numRounds, giver, receiver, transferAmt, playerWealth)
    
    # sanity checks - at this point, should have 1 winner left, and 2 losers
#     assert(winner)
#     assert(numPlayers == 1)
#     assert(len(losers) == NUM_PLAYERS-1)
    
    if printResult:
        reportGameResult(winner, losers, numRounds)
        
    return winner, losers[0], numRounds

In [5]:
playGame(1,2,3, printProgress=True, printResult=True)

Round 1:
Player 2 gave Player 1 $1
Remaining players' wealth: {1: 2, 2: 1, 3: 3}

Round 2:
Player 2 gave Player 1 $1
Remaining players' wealth: {1: 3, 3: 3}

Round 3:
Player 1 gave Player 3 $3
Remaining players' wealth: {3: 6}

Loser 1: 2
Loser 2: 1
Winner: 3
Number of rounds played: 3


(3, 2, 3)

In [6]:
playGame(12,24,36, printProgress=True, printResult=True)

Round 1:
Player 2 gave Player 1 $12
Remaining players' wealth: {1: 24, 2: 12, 3: 36}

Round 2:
Player 3 gave Player 1 $24
Remaining players' wealth: {1: 48, 2: 12, 3: 12}

Round 3:
Player 1 gave Player 2 $12
Remaining players' wealth: {1: 36, 2: 24, 3: 12}

Round 4:
Player 2 gave Player 3 $12
Remaining players' wealth: {1: 36, 2: 12, 3: 24}

Round 5:
Player 1 gave Player 2 $12
Remaining players' wealth: {1: 24, 2: 24, 3: 24}

Round 6:
Player 1 gave Player 3 $24
Remaining players' wealth: {2: 24, 3: 48}

Round 7:
Player 2 gave Player 3 $24
Remaining players' wealth: {3: 72}

Loser 1: 1
Loser 2: 2
Winner: 3
Number of rounds played: 7


(3, 1, 7)

In [7]:
def tallyToDf(winners, losers):
    winnerCount = [(winners.count(i+1)) for i in range(NUM_PLAYERS)]
    loserCount = [(losers.count(i+1)) for i in range(NUM_PLAYERS)]
    data = {"winner": winnerCount, "loser": loserCount}
    return pd.DataFrame.from_dict(data, orient='index', columns=range(1, NUM_PLAYERS+1))

In [8]:
def simGame(n, initState):
    """
    Plays the betting game `n` times, with the provided initial amounts
    """
    x, y, z = initState
    winners = []
    firstLosers = []
    numRoundsPlayed = []
    
    for i in range(n):
        winner, firstLoser, numRounds = playGame(x, y, z)
        winners.append(winner)
        firstLosers.append(firstLoser)
        numRoundsPlayed.append(numRounds)
    
    summaryDf = tallyToDf(winners, firstLosers)
    print(summaryDf)
    player1Won = summaryDf.loc['winner', 1]
    player1Lost = summaryDf.loc['loser', 1]
    print('\n')
    
    print(f"P(winner = Player 1): {player1Won} / {n} = {player1Won / n}")
    print(f"P(loser = Player 1): {player1Lost} / {n} = {player1Lost / n}")
    print(f"Average number of rounds to end game: {np.mean(numRoundsPlayed)}")

In [9]:
simGame(100000, (1,2,3))

            1      2      3
winner  16496  33349  50155
loser   51940  31770  16290


P(winner = Player 1): 16496 / 100000 = 0.16496
P(loser = Player 1): 51940 / 100000 = 0.5194
Average number of rounds to end game: 3.49989


In [10]:
simGame(100000, (12,24,36))

            1      2      3
winner  16581  33116  50303
loser   52234  31744  16022


P(winner = Player 1): 16581 / 100000 = 0.16581
P(loser = Player 1): 52234 / 100000 = 0.52234
Average number of rounds to end game: 3.49319


In [11]:
simGame(100000, (5,7,8))

            1      2      3
winner  24713  35068  40219
loser   44041  32174  23785


P(winner = Player 1): 24713 / 100000 = 0.24713
P(loser = Player 1): 44041 / 100000 = 0.44041
Average number of rounds to end game: 3.76036


In [12]:
25424153614205494181829414445086271633/57961442947535311667794994473863737322

0.4386390731717731